# QAAS 10K: CSS Renderer - Quantum Agent Annealed Swarm

10,000 tasks across the CSS rendering pipeline: Parsing → Style Computation → Layout → Painting → Compositing.

300 agents (240 cloud + 60 local) with token-based cost model and pipeline-stage dependency chains.

This notebook orchestrates the 10K scale quantum routing problem as a stepping stone to the 1M-task browser architecture.

## Setup

Import the CSS Renderer modules:

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from quantum_routing.css_renderer_agents import build_agent_pool, get_agent_stats
from quantum_routing.css_renderer_intents import generate_intents, build_workflow_chains, get_intent_stats
from quantum_routing.css_renderer_model import estimate_problem_size
from quantum_routing.solve_10k_ortools import solve_cpsat, greedy_solve
from quantum_routing.report_10k import print_shift_report, print_comparison, print_pipeline_flow

## Define the Swarm

300 agents across 4 cloud model tiers and 6 local models.
Cloud agents have 50 task capacity each (higher than 1K scale), local agents have 6-10 capacity.

In [2]:
agents, agent_names = build_agent_pool()
stats = get_agent_stats(agents)

print("CSS Renderer Agent Pool (10K Scale)")
print("=" * 50)
print(f"Total agents: {stats['total_agents']}")
print(f"  Cloud: {stats['cloud_agents']} agents, {stats['cloud_capacity']} capacity")
print(f"  Local: {stats['local_agents']} agents, {stats['local_capacity']} capacity")
print(f"Total capacity: {stats['total_capacity']}")

CSS Renderer Agent Pool (10K Scale)
Total agents: 300
  Cloud: 240 agents, 12000 capacity
  Local: 60 agents, 420 capacity
Total capacity: 12420


## Define the Intents

10,000 CSS renderer tasks across 5 pipeline stages:
- **Parsing** (2,500): Tokenizer, parser, AST building
- **Style Computation** (2,500): Selector matching, cascade, inheritance
- **Layout** (2,500): Box model, flexbox, grid, positioning
- **Painting** (1,500): Backgrounds, borders, text, effects
- **Compositing** (1,000): Layers, GPU commands, optimization

In [3]:
intents = generate_intents()
workflow_chains = build_workflow_chains(intents)
stats = get_intent_stats(intents)

print(f"Total tasks: {stats['total']}")
print(f"Total story points: {stats['total_story_points']}")
print(f"Dependent tasks: {stats['dependent_tasks']}")
print(f"Workflow chains: {len(workflow_chains)}")
print()
print("By Stage:")
for stage, count in stats['by_stage'].items():
    print(f"  {stage}: {count}")
print()
print("By Complexity:")
for complexity in ['trivial', 'simple', 'moderate', 'complex', 'very-complex', 'epic']:
    count = stats['by_complexity'].get(complexity, 0)
    print(f"  {complexity}: {count}")

Total tasks: 10000
Total story points: 30675
Dependent tasks: 1260
Workflow chains: 630

By Stage:
  parsing: 2500
  style_computation: 2500
  layout: 2500
  painting: 1500
  compositing: 1000

By Complexity:
  trivial: 1925
  simple: 2825
  moderate: 2875
  complex: 1675
  very-complex: 600
  epic: 100


## Problem Size Analysis

Estimate the problem dimensions. The D-Wave SA approach would need ~118M quadratic terms for dependency penalties — CP-SAT replaces these with ~1,760 linear constraints:

In [4]:
estimates = estimate_problem_size(intents, agents, agent_names)

print("Problem Size Estimates:")
print(f"  Intents: {estimates['num_intents']:,}")
print(f"  Agents: {estimates['num_agents']}")
print(f"  Valid assignments: {estimates['valid_assignments']:,}")
print(f"  Filter rate: {estimates['filter_rate']:.1%}")
print(f"  Dependency edges: {estimates['dependency_edges']}")
print(f"  D-Wave SA would need: ~{estimates['estimated_quadratic_terms']:,.0f} quadratic terms")
print(f"  CP-SAT needs: {estimates['dependency_edges']} linear dependency constraints")
print(f"  Reduction: {estimates['estimated_quadratic_terms'] / max(estimates['dependency_edges'], 1):,.0f}x fewer terms")

Problem Size Estimates:
  Intents: 10,000
  Agents: 300
  Valid assignments: 2,595,500
  Filter rate: 13.5%
  Dependency edges: 1260
  D-Wave SA would need: ~84,881,415 quadratic terms
  CP-SAT needs: 1260 linear dependency constraints
  Reduction: 67,366x fewer terms


## Solve with CP-SAT

OR-Tools CP-SAT replaces D-Wave simulated annealing. Dependencies are modeled as O(dep_edges) linear constraints instead of O(dep_edges × agents²) quadratic terms — no BQM conversion, no memory explosion.
Time budget: 10 minutes.

In [5]:
assignments = solve_cpsat(intents, agents, agent_names)
print(f"\nAssigned {len(assignments)}/{len(intents)} tasks")

Building CP-SAT model: 10000 tasks x 10 model types
  Boolean variables: 67,050
  Model build time: 0.9s

Solving with CP-SAT (time limit: 600s)...

Starting CP-SAT solver v9.15.6755
Parameters: max_time_in_seconds: 600 log_search_progress: true num_workers: 0
Setting number of workers to 8

Initial optimization model '': (model_fingerprint: 0xce8acbe0b49dca6a)
#Variables: 76'650 (#bools: 75'390 #ints: 1'260 in objective) (65'390 primary variables)
  - 75'390 Booleans in [0,1]
  - 1'260 in [0,100]
#kBoolAnd: 16'680 (#enforced: 16'680) (#literals: 33'360)
#kBoolOr: 8'340 (#literals: 25'020)
#kExactlyOne: 10'000 (#literals: 67'050)
#kLinear1: 80
#kLinearN: 1'190 (#terms: 87'020)

Starting presolve at 0.03s
  4.41e-02s  0.00e+00d  [DetectDominanceRelations] 
  9.04e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  3.73e-04s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=9'300 
  7.31e-03s  0.00e+00d  [DetectDuplicateColumns] 
  1.53e-02s  0.00e+

## Factory Floor Shift Report

Full report on task assignments, costs, and pipeline flow:

In [6]:
print_shift_report(assignments, intents, agents, workflow_chains)

  CSS RENDERER FACTORY FLOOR SHIFT REPORT
  10K Tasks - Quantum Agent Annealed Swarm

  Tasks completed:       10000/10000
  Tasks dropped:         0
  Money spent:           $592.32
  Quality targets met:   10000/10000
  Capacity violations:   0
  Dep violations:        0
  Stage order violations: 0

──────────────────────────────────────────────────────────────────────
  PIPELINE STAGE BREAKDOWN
──────────────────────────────────────────────────────────────────────
  Stage                   Tasks       SP         Cost
  ────────────────────────────────────────────────────
  parsing                  2500     5775 $     64.47
  style_computation        2500     7125 $    116.41
  layout                   2500     8625 $    202.95
  painting                 1500     4500 $     63.05
  compositing              1000     4650 $    145.44

──────────────────────────────────────────────────────────────────────
  WORKFLOW CHAIN QUALITY PROGRESSION (sample)
────────────────────────────────────

## Pipeline Flow Analysis

Quality distribution across each CSS pipeline stage:

In [7]:
print_pipeline_flow(assignments, intents, agents)


  CSS PIPELINE FLOW ANALYSIS

  PARSING
    Tasks: 2500/2500 assigned
    Quality distribution:
      basic (<0.6): 80 (3.2%)
      excellent (0.9+): 907 (36.3%)
      fair (0.6-0.8): 119 (4.8%)
      good (0.8-0.9): 1394 (55.8%)

  STYLE_COMPUTATION
    Tasks: 2500/2500 assigned
    Quality distribution:
      excellent (0.9+): 1073 (42.9%)
      fair (0.6-0.8): 1 (0.0%)
      good (0.8-0.9): 1426 (57.0%)

  LAYOUT
    Tasks: 2500/2500 assigned
    Quality distribution:
      excellent (0.9+): 950 (38.0%)
      good (0.8-0.9): 1550 (62.0%)

  PAINTING
    Tasks: 1500/1500 assigned
    Quality distribution:
      basic (<0.6): 100 (6.7%)
      excellent (0.9+): 450 (30.0%)
      good (0.8-0.9): 950 (63.3%)

  COMPOSITING
    Tasks: 1000/1000 assigned
    Quality distribution:
      excellent (0.9+): 200 (20.0%)
      fair (0.6-0.8): 120 (12.0%)
      good (0.8-0.9): 680 (68.0%)


## Greedy Baseline Comparison

Compare CP-SAT optimization against greedy first-come-first-served:

In [8]:
greedy_assignments, greedy_cost = greedy_solve(intents, agents)
print_comparison(assignments, greedy_assignments, greedy_cost, intents, agents)

  HEAD TO HEAD: GREEDY vs QUANTUM ANNEALING
  10K CSS Renderer Task Routing

  Metric                         Greedy          Annealing      
  ──────────────────────────────────────────────────────────────
  Tasks shipped                  9950            10000          
  Tasks dropped                  50              0              
  Money spent                    $892.02         $592.32        
  Story points                   30025           30675          
  Cost per SP                    $0.0297         $0.0193        
  Cloud tasks                    9530            9580           
  Local tasks (free)             420             420            
  Dep violations                 80              0              

  → Annealing shipped 50 more tasks
  → Annealing saved $299.70
  → Annealing delivered 650 more story points
  → Annealing had 80 fewer dependency violations


## Summary

### 10K CSS Renderer Results

| Metric | Target | Actual |
|--------|--------|--------|
| Tasks | 10,000 | - |
| Agents | 300 | - |
| Story Points | 30,675 | - |
| Method | CP-SAT | OR-Tools |

### Next Steps

1. **Scale to 100K**: Increase to 100,000 tasks (intermediate layer)
2. **D-Wave Hybrid**: Test on real quantum hardware
3. **1M Browser**: Full browser architecture with all 5 layers